<a href="https://colab.research.google.com/github/taoyafan/cnn-dailymail/blob/master/make_data_file_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

教程参考：

[想免费用谷歌资源训练神经网络？Colab 详细使用教程](https://jinkey.ai/post/tech/xiang-mian-fei-yong-gu-ge-zi-yuan-xun-lian-shen-jing-wang-luo-colab-xiang-xi-shi-yong-jiao-cheng#toc_2)

[Google Colab Free GPU Tutorial](https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d)

# 重启虚拟机

虚拟机上所有的东西都会消失

In [0]:
# !kill -9 -1

# 授权登录、挂载云盘

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

drive.mount("/content/drive")
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# 赋值必要文件从云盘到 VM

## 文件名和路径设置

是否自动生成测试文件？如果是，则从训练集中抽出一部分作为测试集，否则手动设定测试集

In [0]:
AUTO_TEST = True

if AUTO_TEST:
    TEST_NUM = 10

In [53]:
import time
import os
import shutil

stories_dir = "/content"
split_stories_dir = "/content/split_stories"
tokenized_stories_dir = "/content/tokenized_stories"
finished_files_dir = "/content/finished_files"
drive_finished_files_dir = "/content/drive/My Drive/code/get_to_the_point/finished_files"
smrz_dir = "drive/My Drive/code/get_to_the_point/pointer-generator-master"
chunks_dir = os.path.join(finished_files_dir, "chunked")
drive_chunks_dir = os.path.join(drive_finished_files_dir, "chunked")


train_file_names = ["bytecup.corpus.train.{}.txt".format(i) for i in range(0,9)]
# train_file_names = ["bytecup.corpus.train.0.txt"]
val_file_names = ["bytecup.corpus.validation_set.txt"]
if not AUTO_TEST:
    test_file_names = ["for_test.txt"]

# if not os.path.exists(stories_dir): raise Exception("stories_dir %s doesn't exist." % (stories_dir))
if not os.path.exists(split_stories_dir): os.makedirs(split_stories_dir)
if not os.path.exists(tokenized_stories_dir): os.makedirs(tokenized_stories_dir)
if not os.path.exists(finished_files_dir): os.makedirs(finished_files_dir)
if not os.path.exists(drive_finished_files_dir): os.makedirs(drive_finished_files_dir)
if not os.path.exists(chunks_dir): os.makedirs(chunks_dir)
# if not os.path.exists(drive_chunks_dir): os.makedirs(drive_chunks_dir)
if not os.path.exists(smrz_dir): raise Exception("smrz_dir %s doesn't exist." % (smrz_dir))
    
print("train_file_names: %s"% train_file_names)
print("val_file_names: %s"% val_file_names)
if not AUTO_TEST:
    print("test_file_names: %s"% test_file_names)
else:
    print("test_file_names: %s"% train_file_names)

train_file_names: ['bytecup.corpus.train.0.txt', 'bytecup.corpus.train.1.txt', 'bytecup.corpus.train.2.txt', 'bytecup.corpus.train.3.txt', 'bytecup.corpus.train.4.txt', 'bytecup.corpus.train.5.txt', 'bytecup.corpus.train.6.txt', 'bytecup.corpus.train.7.txt', 'bytecup.corpus.train.8.txt']
val_file_names: ['bytecup.corpus.validation_set.txt']
test_file_names: ['bytecup.corpus.train.0.txt', 'bytecup.corpus.train.1.txt', 'bytecup.corpus.train.2.txt', 'bytecup.corpus.train.3.txt', 'bytecup.corpus.train.4.txt', 'bytecup.corpus.train.5.txt', 'bytecup.corpus.train.6.txt', 'bytecup.corpus.train.7.txt', 'bytecup.corpus.train.8.txt']


## 复制数据原始文件（file_names）

In [29]:
drive_dir = "drive/My Drive/code"

def copy_file(file_names, drive_dir):
    for file_name in file_names:
        src_file = os.path.join(drive_dir, file_name)
        print("Copy file from \"%s\" to \"%s\""% (src_file, file_name))
        shutil.copyfile(src_file, file_name) 

copy_file(train_file_names, drive_dir)
copy_file(val_file_names, drive_dir)
if not AUTO_TEST:
    copy_file(test_file_names, drive_dir)


Copy file from "drive/My Drive/code/bytecup.corpus.train.0.txt" to "bytecup.corpus.train.0.txt"
Copy file from "drive/My Drive/code/bytecup.corpus.train.1.txt" to "bytecup.corpus.train.1.txt"
Copy file from "drive/My Drive/code/bytecup.corpus.train.2.txt" to "bytecup.corpus.train.2.txt"
Copy file from "drive/My Drive/code/bytecup.corpus.train.3.txt" to "bytecup.corpus.train.3.txt"
Copy file from "drive/My Drive/code/bytecup.corpus.train.4.txt" to "bytecup.corpus.train.4.txt"
Copy file from "drive/My Drive/code/bytecup.corpus.train.5.txt" to "bytecup.corpus.train.5.txt"
Copy file from "drive/My Drive/code/bytecup.corpus.train.6.txt" to "bytecup.corpus.train.6.txt"
Copy file from "drive/My Drive/code/bytecup.corpus.train.7.txt" to "bytecup.corpus.train.7.txt"
Copy file from "drive/My Drive/code/bytecup.corpus.train.8.txt" to "bytecup.corpus.train.8.txt"
Copy file from "drive/My Drive/code/bytecup.corpus.validation_set.txt" to "bytecup.corpus.validation_set.txt"


## 原始方法，不需要挂载云盘

显示云盘根目录的文件

In [0]:
# file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
# for file1 in file_list:
#       print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))


打开 code 文件

In [0]:
# file_list = drive.ListFile({'q': "'18ijhbBiNtaqpgiNwzxAf6a3W13Pc8qSa' in parents and trashed=false"}).GetList()
# for file1 in file_list:
#       print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))
  

读取云盘中的文件内容

In [0]:
def get_file_content(file_name, show = False):
    id = [file['id'] for file in file_list if file['title'] == file_name][0]
  
    if id == []:
        print("There are not a file named %s"% file_name)
        return []
  
    print("Matched file named %s, and the id is %s"% (file_name, id))
    file = drive.CreateFile({'id': id})
    txt = file.GetContentString()
    if show:
        print("This is the content of file %s :\n\n"% file_name)
        print(txt)
    
    print("Read file complete, length of file is %d" % len(txt))
    return txt
  
# txt = get_file_content(file_name, show = False)

内容写入虚拟机

In [0]:
def write_file(file_name, lines):
    with open(file_name, 'w') as f:
        for line in lines:
            f.write(line)


读取根目录文件，测试写入成功

In [0]:
def show_file(file_name):
    with open(file_name, 'r') as load_f:
        print("This is the content of file %s :\n\n"% file_name)
        lines = load_f.readlines()
        for i, line in enumerate(lines):
            print(line)
        print("\n\n")
        return lines
    
# show_file(file_name)

删除文件，例如

remove_files(os.listdir(dir))

可以删除路径dir下所有文件

In [0]:
def remove_files(file_names):
    for file_name in file_names:
        os.remove(file_name)

# 数据预处理

## 将每个数据集文件拆分个多个小文件
每个小文件包含一个故事的内容和标题，以 ***数据类型_ ID***  命名，如 ***train_1*** 存放在 split_stories_dir 下

In [0]:
import json
import os
import re
import random

def split_lines(lines, story_type):
    '''
        输入参数 lines 是多行的数据文件，每行是一个样本，样本类型为 story_type
        
        将输入的 lines 按行拆分，分别写入文件，文件名为 story_type_id
    '''
    print('Writting %s file in %s...'% (story_type, split_stories_dir))
    
    # 读取数据集的每一个样本
    for i, line in enumerate(lines):
        txt_dic = json.loads(line)
        split_file_dir = os.path.join(split_stories_dir, "%s_%s"%(story_type, txt_dic['id']))

        # 将每个样本单独写入一个文件中
        with open(split_file_dir, "w") as f:
            txt_dic['content'] = re.sub(r"(?<=[a-zA-Z\"\'\[\]\(\)\-\$]{2})[.](?=\S)|[.](?=[a-zA-Z\"\'\[\]\(\)\-\$]{2})",
                                        ". ", txt_dic['content']) 
            f.write(txt_dic['content'])
            f.write(' . ')
            if('title' in txt_dic.keys()):
                f.write("\n\n@title.\n")
                f.write(txt_dic['title'])


def split_story_files(file_names, file_type):
    
    
    for name in file_names:
        file_dir = os.path.join(stories_dir, name)
        heads = []
        desc = []

        print('Opening file: %s...' % name)
        time_start = time.time()

        # 打开每一个数据集
        with open(file_dir, 'r') as load_f:
            lines = load_f.readlines()
        
        if AUTO_TEST and file_type == 'train':
#             random.shuffle(lines)
            split_lines(lines[: TEST_NUM], 'test')
            split_lines(lines[TEST_NUM : -1], 'train')
        else:
            split_lines(lines, file_type)

        print("completed")
        time_end = time.time()
        print('Consume time:', time_end - time_start, '\n')

In [37]:
# shutil.rmtree(split_stories_dir)
# os.makedirs(split_stories_dir)

split_story_files(train_file_names, "train")
split_story_files(val_file_names, "val")

if not AUTO_TEST:
    split_story_files(test_file_names, "test")
    


Opening file: bytecup.corpus.train.0.txt...
Writting test file in split_stories...
Writting train file in split_stories...
completed
Consume time: 55.81170415878296 

Opening file: bytecup.corpus.train.1.txt...
Writting test file in split_stories...
Writting train file in split_stories...
completed
Consume time: 57.35386276245117 

Opening file: bytecup.corpus.train.2.txt...
Writting test file in split_stories...
Writting train file in split_stories...
completed
Consume time: 57.28793811798096 

Opening file: bytecup.corpus.train.3.txt...
Writting test file in split_stories...
Writting train file in split_stories...
completed
Consume time: 58.22538375854492 

Opening file: bytecup.corpus.train.4.txt...
Writting test file in split_stories...
Writting train file in split_stories...
completed
Consume time: 61.12573981285095 

Opening file: bytecup.corpus.train.5.txt...
Writting test file in split_stories...
Writting train file in split_stories...
completed
Consume time: 59.69430065155029 

## 对每篇文章进行分词（分句）

### 安装 java 添加环境变量

In [0]:
time_start = time.time()

!apt-get install openjdk-8-jdk
!java -version
os.environ['CLASSPATH']="/content/drive/My Drive/pack/stanford-corenlp-full-2018-10-05/stanford-corenlp-3.9.2.jar"

time_end = time.time()
print('Consume time:', time_end - time_start)

### 选择是否分句
若为 DIVIDE_SENTENCE = True 则分句加分词，否则只分词

In [0]:
DIVIDE_SENTENCE = False

### 只分词不分句
先生成映射文件 mapping，再使用 PTBTokenizer 对文件进行直接操作

In [21]:
import copy
stories = os.listdir(split_stories_dir)
print(stories[9*stories_len//10])

time_start = time.time()
sub_stories = copy.deepcopy(stories[9*stories_len//10 : 1+(10+1)*stories_len//10])
print('Consume time:', time.time() - time_start)

print(sub_stories[0])
# sub_stories[0] = 5
print(stories[9*stories_len//10])
print(sub_stories[0])

print("id s: %d, id sub: %d"% (id(stories[9*stories_len//10]), id(sub_stories[0])))

train_1209789
Consume time: 0.005620002746582031
train_1209789
train_1209789
train_1209789
id s: 139741915877040, id sub: 139741915877040


In [38]:

import subprocess

if not DIVIDE_SENTENCE:

    
    stories = os.listdir(split_stories_dir)
    stories_len = len(stories)
    for i in range(10):
        time_start = time.time()
        print("Processing %d to %d of %d..."% (i*stories_len//10, 1+(i+1)*stories_len//10, stories_len))
        sub_stories = copy.deepcopy(stories[i*stories_len//10 : 1+(i+1)*stories_len//10])
        time_end = time.time()
        print('Making a copy consume time:', time_end - time_start)
        
        #生成映射文件 mapping.txt
        print("Making list of files to tokenize...")
        with open("mapping.txt", "w") as f:
            time_end = time.time()
            print('opening mapping.txt consume time:', time_end - time_start)
            for s in sub_stories:
                f.write("%s \t %s\n" % (os.path.join(split_stories_dir, s), os.path.join(tokenized_stories_dir, s)))
        time_end = time.time()
        print('Made list consume time:', time_end - time_start)

        print("Token file frome \"%s\" to \"%s\""% (split_stories_dir, tokenized_stories_dir))

        command = ['java', 'edu.stanford.nlp.process.PTBTokenizer', '-ioFileList', '-preserveLines', 'mapping.txt']
        subprocess.call(command)


        time_end = time.time()
        print('Token consume time:', time_end - time_start, "\n\n")
        os.remove("mapping.txt")

Processing 0 to 126282 of 1262810...
Making a copy consume time: 0.11025190353393555
Making list of files to tokenize...
opening mapping.txt consume time: 0.11122441291809082
Made list consume time: 0.5680255889892578
Token file frome "split_stories" to "tokenized_stories"
Token consume time: 160.12184500694275 


Processing 126281 to 252563 of 1262810...
Making a copy consume time: 0.1186068058013916
Making list of files to tokenize...
opening mapping.txt consume time: 0.11981964111328125
Made list consume time: 0.5682628154754639
Token file frome "split_stories" to "tokenized_stories"
Token consume time: 172.49692368507385 


Processing 252562 to 378844 of 1262810...
Making a copy consume time: 0.11917591094970703
Making list of files to tokenize...
opening mapping.txt consume time: 0.1201620101928711
Made list consume time: 0.5700123310089111
Token file frome "split_stories" to "tokenized_stories"
Token consume time: 177.52065587043762 


Processing 378843 to 505125 of 1262810...
Ma

### 分句加分词
无法利用映射文件进行分词，有BUG，故只能一个一个文件分词

In [0]:
import os
if DIVIDE_SENTENCE:
    
    time_start = time.time()
    
    stories = os.listdir(split_stories_dir)
    for s in stories:
        os.system("java edu.stanford.nlp.process.DocumentPreprocessor -preserveLines < %s > %s" %
                  (os.path.join(split_stories_dir, s), os.path.join(tokenized_stories_dir, s)))

    time_end = time.time()
    print('Consume time:', time_end - time_start)

## 格式化文章并存储

### （函数）根据文件地址读文件，返回行列表

In [0]:
def read_text_file(text_file):
    lines = []
    with open(text_file, "r") as f:
        for line in f:
            lines.append(line.strip())
        return lines

### （函数）根据文件内容获得文章和摘要

In [0]:
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'

def get_art_abs(story_file):
    lines = read_text_file(story_file)

    # Lowercase everything
    lines = [line.lower() for line in lines]

    # Separate out article and abstract sentences
    article_lines = []
    titles = []
    next_is_title = False
    for idx,line in enumerate(lines):
        if line == "":
            continue # empty line
        elif line.startswith("@title"):
            next_is_title = True
        elif next_is_title:
            titles.append(line)
        else:
            article_lines.append(line)

    # Make article into a single string
    article = ' '.join(article_lines)

    # Make abstract into a signle string, putting <s> and </s> tags around the sentences
    title = ' '.join(["%s %s %s" % (SENTENCE_START, sent, SENTENCE_END) for sent in titles])

    return article, title

### 写入 bin 文件

In [0]:
from tensorflow.core.example import example_pb2
import struct
import glob

def write_to_bin(src_dir, out_dir, file_type):

    time_start = time.time()

    out_file = os.path.join(out_dir, '%s.bin'% file_type)
    print("Writing file %s.bin to %s..."% (file_type, out_dir) )
    with open(out_file, 'wb') as writer:
        stories = glob.glob(os.path.join(tokenized_stories_dir, '%s_*'% file_type))
        
        if file_type == 'val':
            stories.sort(key=lambda str: int(re.search('(?<=val_).*', str).group()))
            print(stories)
        

        for idx, s in enumerate(stories):
            article, title = get_art_abs(s)
#             print("article: \n%s\n\n"%article)
#             print("title: \n%s\n" % title)

            tf_example = example_pb2.Example()
            tf_example.features.feature['article'].bytes_list.value.extend([article.encode()])
            tf_example.features.feature['abstract'].bytes_list.value.extend([title.encode()])
            tf_example_str = tf_example.SerializeToString()

            str_len = len(tf_example_str)
            writer.write(struct.pack('q', str_len))
            writer.write(struct.pack('%ds' % str_len, tf_example_str))

    time_end = time.time()
    print('Consume time:', time_end - time_start)


In [42]:
write_to_bin(tokenized_stories_dir, finished_files_dir, 'train')
write_to_bin(tokenized_stories_dir, finished_files_dir, 'test')
write_to_bin(tokenized_stories_dir, finished_files_dir, 'val')

Writing file train.bin to finished_files...
Consume time: 368.86140513420105
Writing file test.bin to finished_files...
Consume time: 2.3707525730133057
Writing file val.bin to finished_files...
['tokenized_stories/val_1', 'tokenized_stories/val_2', 'tokenized_stories/val_3', 'tokenized_stories/val_4', 'tokenized_stories/val_5', 'tokenized_stories/val_6', 'tokenized_stories/val_7', 'tokenized_stories/val_8', 'tokenized_stories/val_9', 'tokenized_stories/val_10', 'tokenized_stories/val_11', 'tokenized_stories/val_12', 'tokenized_stories/val_13', 'tokenized_stories/val_14', 'tokenized_stories/val_15', 'tokenized_stories/val_16', 'tokenized_stories/val_17', 'tokenized_stories/val_18', 'tokenized_stories/val_19', 'tokenized_stories/val_20', 'tokenized_stories/val_21', 'tokenized_stories/val_22', 'tokenized_stories/val_23', 'tokenized_stories/val_24', 'tokenized_stories/val_25', 'tokenized_stories/val_26', 'tokenized_stories/val_27', 'tokenized_stories/val_28', 'tokenized_stories/val_29', '

In [0]:
# stories = os.listdir(tokenized_stories_dir)
# remove_files(os.listdir(tokenized_stories_dir))
# stories = os.listdir(tokenized_stories_dir)


### 将文件分块，每个块的文章个数为 CHUNK_SIZE

In [43]:
CHUNK_SIZE = 5000 # num examples per chunk, for the chunked data 1000

def chunk_file(set_name):
        
    in_file = os.path.join(finished_files_dir, '%s.bin' % set_name)
    
    if not os.path.exists(in_file):
        print("File  %s not exist"% in_file)
        return
    
    with open(in_file, 'rb') as reader:
        chunk = 0
        finished = False
        while not finished:
            chunk_fname = os.path.join(chunks_dir, '%s_%03d.bin' % (set_name, chunk)) # new chunk
            with open(chunk_fname, 'wb') as writer:
                for _ in range(CHUNK_SIZE):
                    len_bytes = reader.read(8)
                    if not len_bytes:
                        finished = True
                        break
                    str_len = struct.unpack('q', len_bytes)[0]
                    example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
                    writer.write(struct.pack('q', str_len))
                    writer.write(struct.pack('%ds' % str_len, example_str))
                chunk += 1
            
def chunk_all():
    # Chunk the data
    for set_name in ['train', 'val', 'test']:
        time_start = time.time()
        print("Splitting %s data into chunks..." % set_name)
        chunk_file(set_name)
        
        time_end = time.time()
        print('Consume time:', time_end - time_start)
    print("Saved chunked data in %s" % chunks_dir)
    
# 分块
chunk_all()

Splitting train data into chunks...
Consume time: 56.73035001754761
Splitting val data into chunks...
Consume time: 0.02121114730834961
Splitting test data into chunks...
Consume time: 0.007974863052368164
Saved chunked data in finished_files/chunked


## 替换虚拟机的文件到云盘中

### （函数）复制目录 A 下文件到目录 B

In [0]:
def copy_files(src_dir, des_dir):
    names = os.listdir(src_dir)
    file_num = len(names)
    time_start = time.time()
    
    for num, name in enumerate(names):
        
        src_file = os.path.join(src_dir, name)
        des_file = os.path.join(des_dir, name)
        shutil.copyfile(src_file, des_file) 
        print("\rCopied %d/%d files"% (num, file_num), end='')
        
    time_end = time.time()
    print('Consume time:', time_end - time_start, "\n\n")
    
    print("Copied %d files from \n%s \nto \n%s"% (file_num, src_dir, des_dir))
    

### 将 VM 的 chunk 文件赋值到云盘

In [0]:
if drive_chunks_dir != chunks_dir :
    if os.path.exists(drive_chunks_dir): 
        shutil.rmtree(drive_chunks_dir)
        
    os.makedirs(drive_chunks_dir)
    copy_files(chunks_dir, drive_chunks_dir)

# 运行算法程序

## 通用程序

### 设置通用路径、参数

In [0]:
vocab_path = "/content/drive/My Drive/code/get_to_the_point/finished_files/vocab"
VM_log_root = "/content/log"
drive_log_root = "/content/drive/My Drive/code/get_to_the_point/log"
exp_name = "data0_train"
single_pass = "True"
batch_size = "256"
summarization_file = os.path.join(smrz_dir, "run_summarization.py")

log_root = VM_log_root
exp_dir = os.path.join(log_root, exp_name)
drive_exp_dir = os.path.join(drive_log_root, exp_name)

drive_train_dir = "/content/drive/My Drive/code/get_to_the_point/log/train"


if not os.path.exists(log_root): os.makedirs(log_root)

安装库  pyrough

In [56]:
!pip install pyrouge

    100% |████████████████████████████████| 61kB 1.1MB/s 
  Running setup.py bdist_wheel for pyrouge ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/75/d3/0c/e5b04e15b6b87c42e980de3931d2686e14d36e045058983599
Successfully built pyrouge


### （函数）执行 shell 命令

输入和在 shell 相同的命令，如路径中的空格前需要加"\"

In [0]:
import subprocess

MAX_MEG_LEN = 5000

def shell_command(str):
    
    too_long = False
    
    print(str)
    try:
        out_bytes = subprocess.check_output(str, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        print(len(e.output))
        if(len(e.output) > MAX_MEG_LEN):
            too_long = True
            out_bytes_begin = e.output[:MAX_MEG_LEN//2]       # Output generated before error
            print(out_bytes_begin)
            out_bytes_end = e.output[-MAX_MEG_LEN//2:]       # Output generated before error
        else:
            out_bytes = e.output
        code      = e.returncode   # Return code

        
    if too_long:
        out_text = out_bytes_begin.decode('utf-8') + "\n...\n...\n...\n" + out_bytes_end.decode('utf-8')
    else:
        out_text = out_bytes.decode('utf-8')
    print(out_text)

### （函数）得到可执行命令

In [0]:
def get_command():
    if(mode != "decode"):
        sp = "False"
    else:
        sp = single_pass
        
    return ("python3 %s --mode=%s --data_path=%s --vocab_path=%s --log_root=%s --exp_name=%s --single_pass=%s --batch_size=%s" 
                  % (summarization_file.replace(' ', '\ '),
                     mode, data_path.replace(' ', '\ '),
                     vocab_path.replace(' ', '\ '),
                     log_root.replace(' ', '\ '),
                     exp_name,
                     sp,
                     batch_size))

## 训练

### 配置参数

In [0]:
mode = "train"
data_path = os.path.join(chunks_dir, "train_*")

### 复制 exp 文件到当前 exp

In [0]:
shutil.rmtree(exp_dir)

In [91]:
time_start = time.time()

if not os.path.exists(exp_dir):
    if os.path.exists(drive_exp_dir):
        shutil.copytree(drive_exp_dir, exp_dir)
    else:
        os.makedirs(exp_dir)
        shutil.copytree(os.path.join(drive_log_root, "train"), os.path.join(exp_dir, "train"))

time_end = time.time()
print('Consume time:', time_end - time_start)


Consume time: 39.12252712249756


### 执行命令
产生可执行的命令，复制在下一个 cell 执行即可

In [88]:
print("!%s"% get_command())

!python3 drive/My\ Drive/code/get_to_the_point/pointer-generator-master/run_summarization.py --mode=train --data_path=/content/finished_files/chunked/train_* --vocab_path=/content/drive/My\ Drive/code/get_to_the_point/finished_files/vocab --log_root=/content/log --exp_name=data0_train --single_pass=False --batch_size=256


In [0]:
while True:
    time_start = time.time()
    
    shell_command(get_command())
    
    print("\n\nError,Running time: %d s"% time.time()-time_start)
    lines = show_file(os.path.join(exp_dir, "train/checkpoint"))
    lines[0] = lines[0][:-8] + lines[2][-8:-2] + lines[0][-2:]
    write_file(os.path.join(exp_dir, "train/checkpoint"), lines)
    lines = show_file(os.path.join(exp_dir, "train/checkpoint"))
    print("\n\nChange checkpoint\n\n\n")
    
    if os.path.exists(drive_exp_dir): 
        shutil.rmtree(drive_exp_dir)

    print(shutil.copytree(exp_dir, drive_exp_dir))
    
while True:
    pass

# !python3 drive/My\ Drive/code/get_to_the_point/pointer-generator-master/run_summarization.py --mode=train --data_path=/content/finished_files/chunked/train_* --vocab_path=/content/drive/My\ Drive/code/get_to_the_point/finished_files/vocab --log_root=/content/log --exp_name=data0_train --single_pass=False --batch_size=256

### 出现 NAN 更改 checkpoint 文件

更改当前的 checkpoint 为第二新

In [84]:
lines = show_file(os.path.join(exp_dir, "train/checkpoint"))
lines[0] = lines[0][:-5] + lines[2][-5:-2] + lines[0][-2:]
write_file(os.path.join(exp_dir, "train/checkpoint"), lines)
lines = show_file(os.path.join(exp_dir, "train/checkpoint"))

This is the content of file /content/drive/My Drive/code/get_to_the_point/log/data0_train/train/checkpoint :


model_checkpoint_path: "/content/log/data0_train/train/model.ckpt-248638"

all_model_checkpoint_paths: "/content/log/data0_train/train/model.ckpt-248617"

all_model_checkpoint_paths: "/content/log/data0_train/train/model.ckpt-248638"

all_model_checkpoint_paths: "/content/log/data0_train/train/model.ckpt-248659"




This is the content of file /content/drive/My Drive/code/get_to_the_point/log/data0_train/train/checkpoint :


model_checkpoint_path: "/content/log/data0_train/train/model.ckpt-248617"

all_model_checkpoint_paths: "/content/log/data0_train/train/model.ckpt-248617"

all_model_checkpoint_paths: "/content/log/data0_train/train/model.ckpt-248638"

all_model_checkpoint_paths: "/content/log/data0_train/train/model.ckpt-248659"






## TEST 解码

### 配置 pyrough

In [0]:
drive_ROUGH_dir = "drive/My\ Drive/pack/pyrouge-master.zip"
ROUGH_dir = "pack/pyrouge-master/tools/ROUGE-1.5.5"

shell_command("unzip -o %s -d pack"% drive_ROUGH_dir)
shell_command("pyrouge_set_rouge_path %s"% ROUGH_dir)
shell_command("rm %s/data/WordNet-2.0.exc.db"% ROUGH_dir)

cmd0 = "%s/data/WordNet-2.0-Exceptions/buildExeptionDB.pl"% ROUGH_dir
cmd1 = "./%s/data/WordNet-2.0-Exceptions"% ROUGH_dir
cmd2 = "./%s/data/smart_common_words.txt"% ROUGH_dir
cmd3 = "./%s/data/WordNet-2.0.exc.db"% ROUGH_dir
shell_command("perl %s %s %s %s"% (cmd0, cmd1, cmd2, cmd3))

!cpan install XML::DOM

### 配置参数

In [121]:
mode = "decode"
data_path = os.path.join(chunks_dir, "test_*")


!python3 drive/My\ Drive/code/get_to_the_point/pointer-generator-master/run_summarization.py --mode=decode --data_path=/content/finished_files/chunked/test_* --vocab_path=/content/drive/My\ Drive/code/get_to_the_point/finished_files/vocab --log_root=/content/log --exp_name=data0_train --single_pass=True --batch_size=256


删除解码文件

In [0]:
# shutil.rmtree(os.path.join(exp_dir, "decode_test_400maxenc_4beam_5mindec_15maxdec_ckpt-247405"))

### 执行命令
产生可执行的命令，复制在下一个 cell 执行即可

In [0]:
print("!%s"% get_command())

In [149]:
!python3 drive/My\ Drive/code/get_to_the_point/pointer-generator-master/run_summarization.py --mode=decode --data_path=/content/finished_files/chunked/test_* --vocab_path=/content/drive/My\ Drive/code/get_to_the_point/finished_files/vocab --log_root=/content/log --exp_name=data0_train --single_pass=True --batch_size=256

INFO:tensorflow:Starting seq2seq_attention in decode mode...








max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: cardholders
HParams(mode='decode', hidden_dim=256, emb_dim=128, batch_size=4, max_enc_steps=400, max_dec_steps=15, lr=0.15, adagrad_init_acc=0.1, rand_unif_init_mag=0.02, trunc_norm_init_std=0.0001, max_grad_norm=2.0, pointer_gen=True, coverage=False, cov_loss_wt=1.0)
INFO:tensorflow:Building graph...
example_generator completed reading all datafiles. No more data.
INFO:tensorflow:The example generator for this example queue filling thread has exhausted data.
INFO:tensorflow:single_pass mode is on, so we've finished reading dataset. This thread is stopping.
INFO:tensorflow:Adding attention_decoder timestep 0 of 1
INFO:tensorflow:Time to build graph: 0 seconds
2018-11-15 08:30:12.499552: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA no

## VAL 解码生成 result.zip

### 配置参数

In [193]:
mode = "decode"
data_path = os.path.join(chunks_dir, "val_*")


!python3 drive/My\ Drive/code/get_to_the_point/pointer-generator-master/run_summarization.py --mode=decode --data_path=/content/finished_files/chunked/val_* --vocab_path=/content/drive/My\ Drive/code/get_to_the_point/finished_files/vocab --log_root=/content/log --exp_name=data0_train --single_pass=True --batch_size=256


### 执行命令
产生可执行的命令，复制在下一个 cell 执行即可

In [51]:
print("!%s"% get_command())

NameError: ignored

In [194]:
!python3 drive/My\ Drive/code/get_to_the_point/pointer-generator-master/run_summarization.py --mode=decode --data_path=/content/finished_files/chunked/val_* --vocab_path=/content/drive/My\ Drive/code/get_to_the_point/finished_files/vocab --log_root=/content/log --exp_name=data0_train --single_pass=True --batch_size=256

INFO:tensorflow:Starting seq2seq_attention in decode mode...








max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: cardholders
HParams(mode='decode', hidden_dim=256, emb_dim=128, batch_size=4, max_enc_steps=400, max_dec_steps=15, lr=0.15, adagrad_init_acc=0.1, rand_unif_init_mag=0.02, trunc_norm_init_std=0.0001, max_grad_norm=2.0, pointer_gen=True, coverage=False, cov_loss_wt=1.0)
INFO:tensorflow:Building graph...
INFO:tensorflow:Adding attention_decoder timestep 0 of 1
INFO:tensorflow:Time to build graph: 0 seconds
2018-11-15 11:30:18.983980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-11-15 11:30:18.984488: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 0 with properties: 
name: Tesla K80 major: 3 mino

### 创建 result 文件

In [0]:
result_dir = "result"
if os.path.exists(result_dir): 
    os.system("rm %s -rf"% result_dir)
    
os.mkdir(result_dir)

### 复制结果到 result 文件夹，并压缩

In [201]:
decoded_files_dir = "log/data0_train/decode_val_400maxenc_4beam_5mindec_15maxdec_ckpt-248659/decoded"

decodeed_files = os.listdir(decoded_files_dir)
decodeed_files.sort()
print(decodeed_files)

# 循环读取 decode 文件，重命名写入 result 文件中
for i, fname in enumerate(decodeed_files):
    out_name = "%d.txt" % (i+1)
    print("Out name is : ", out_name)
    shutil.copyfile(os.path.join(decoded_files_dir, fname), os.path.join(result_dir, out_name))

!zip -r result.zip result

['000000_decoded.txt', '000001_decoded.txt', '000002_decoded.txt', '000003_decoded.txt', '000004_decoded.txt', '000005_decoded.txt', '000006_decoded.txt', '000007_decoded.txt', '000008_decoded.txt', '000009_decoded.txt', '000010_decoded.txt', '000011_decoded.txt', '000012_decoded.txt', '000013_decoded.txt', '000014_decoded.txt', '000015_decoded.txt', '000016_decoded.txt', '000017_decoded.txt', '000018_decoded.txt', '000019_decoded.txt', '000020_decoded.txt', '000021_decoded.txt', '000022_decoded.txt', '000023_decoded.txt', '000024_decoded.txt', '000025_decoded.txt', '000026_decoded.txt', '000027_decoded.txt', '000028_decoded.txt', '000029_decoded.txt', '000030_decoded.txt', '000031_decoded.txt', '000032_decoded.txt', '000033_decoded.txt', '000034_decoded.txt', '000035_decoded.txt', '000036_decoded.txt', '000037_decoded.txt', '000038_decoded.txt', '000039_decoded.txt', '000040_decoded.txt', '000041_decoded.txt', '000042_decoded.txt', '000043_decoded.txt', '000044_decoded.txt', '000045_d

## 复制 log 文件和 result.zip 到云盘
必须手动复制，如果自动保存云盘，之前覆盖掉的文件会进入回收站而不会被删除

In [1]:
if os.path.exists(drive_exp_dir): 
    shutil.rmtree(drive_exp_dir)

print(shutil.copytree(exp_dir, drive_exp_dir))

if os.path.exists("result.zip"):
    drive_result_dir = os.path.join(drive_dir, "result.zip")
    if os.path.exists(drive_result_dir): 
        os.remove(drive_result_dir)

    print(shutil.copyfile("result.zip", drive_result_dir))

NameError: ignored